In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import Select

from fuzzywuzzy import fuzz
from time import sleep
import operator


In [59]:

def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser


In [60]:

def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name


In [61]:
data = None
import pandas as pd
data = pd.read_excel('DecRenewalAccounts_V.08_ReportID.xlsx',sheet_name = 'Intellect DEC ReportID')
data.shape



(292, 17)

In [62]:
data['Account_Name2'][0]

'MASSACHUSETTS HOUSING FINANCE AGENCY'

In [51]:
data.keys()

Index(['LM Sr. No.', 'Account_Name', 'Street', 'City', 'State', 'Zip',
       'Country', 'LOB', 'Renewal Date', 'Cleaned Account_Name',
       'Cleaned Street', 'Cleaned City', 'Cleaned State', 'Cleaned Zip',
       'Cleaned Country', 'Melissa Concatenated Company & Addresses',
       'Vlookup P to match R', 'Deba Concatenated Company & Addresss',
       'Column24', 'Account_Name2', 'Street3', 'City4', 'State5', 'Zip6'],
      dtype='object')

In [63]:

names=data['Cleaned Updated Account_Name'].tolist()
# locations = data['Cleaned Country'].tolist()

original = data['Cleaned Updated Account_Name'].tolist()
query_company = names
query_country = ['USA']*len(query_company)

driver = get_browser()


C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [64]:

# driver= webdriver.Chrome('chromedriver')
driver = get_browser()
url = "https://www.indeed.com/companies"

# query_company= ['THE SOUTHERN POVERTY LAW CENTER INC','AT&T']
# query_country = ['USA','USA']

company_name=[]
review_link =[]
score_max =[]
original_name=[]

for i in range(0,len(query_company)):
    
    
    sleep(2)
    driver.get(url)

    ## submitting name & location
    co_name = driver.find_element_by_xpath("""//*[@id="exploreCompaniesWhat"]""")
    co_name.send_keys(query_company[i].lower())

    location = driver.find_element_by_xpath("""//*[@id="exploreCompaniesWhere"]""")
    location.send_keys(query_country[i].lower())

    driver.find_element_by_xpath("""//*[@id="exploreCompaniesSearchFormContainer"]/form/div[3]/button""").click()

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:
        ### appending list of company names
        name =[]
        for l in soup.find_all("a",{"class":"cmp-CompanyWidget-name"}):
            name.append(l.text.strip())
        print(name)
        
        ## storing list of companies fuzzy score with the query company name
        score=[]
        for s in name:
            score.append(fuzz.ratio(clean(query_company[i]).lower(),clean(s).lower()))
#         print(score)
        
        ## cut off as 90-- if more than one cutoff is 90-- choosing max value. What if max value appears more than one time??
        values={}
        for k,v in enumerate(score):
            if v>79:
                values[k] = v
        
#         print(values)
        
        position = max(values.items(), key=operator.itemgetter(1))[0]
#         print(position)
        
        score_maxvalue = max(values.items(), key=operator.itemgetter(1))[1]
#         print(score_maxvalue)
         
        NAME = name[position]
        ORI_NAME = original[i]
        LINK = "https://www.indeed.com"+soup.find_all("a",{"data-tn-element":"reviews-footer-link"},href=True)[position]['href']
        SCORE = score_maxvalue
    except:
        NAME=query_company[i]
        ORI_NAME = original[i]
        LINK=""  
        SCORE =""
    company_name.append(NAME)
    review_link.append(LINK)
    score_max.append(SCORE)
    original_name.append(ORI_NAME)
    
    driver.refresh 

driver.quit()



C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


['Massachusetts Housing Finance Agency']
['Fleetwood aluminum products']
['U.S. Army', 'Peopleready', 'U.S. Navy', 'Comcast', 'Tradesmen International', 'McCarthy Building Co', 'State of Arizona', 'NPL Construction', 'AECOM', 'CRH', 'Rogers', 'CDM Smith', 'Ultimate LLC', 'Construction Labor Contractors', 'intertek', 'Jacobs', 'Advantage Engineers, LLC', 'Match Marketing', 'KGPCo', 'Habitat For Humanity', 'Fluor Corp.', 'Parsons', 'Signature Retail Services', 'Tremco Incorporated', 'Apex Companies, LLC', 'Blackmon Mooring & BMSCAT', 'Pacific Gas and Electric Company (PG&E)', 'Permabilt Industries, Inc.', 'Miller Pipeline LLC', 'AT&T', 'Edison Energy', 'DuPont', 'Installed Building Products', 'Staff Zone', 'Landscapes Unlimited LLC', 'IBEW', 'Holtger Bros', 'Wood Group', 'Champion Window', 'Alpha EMC', 'HDR', 'CPI Security Systems', 'Burns & McDonnell', 'Altice USA', 'Mortenson Construction', 'Bear Staffing Services', 'MasTec Utility Services', 'NiSource', 'Symbia Logistics', 'Flatiron C

['Sound Around', 'All Around Sound Entertainment']
['World Class Flowers LLC', 'world class flowers']
[]
['Braun Linen Service Inc']
['AT&T', 'Wells Fargo', 'United States Postal Service', 'IBM', 'Bank of America', 'Uber Partner Drivers', 'Comcast', 'Verizon', 'FedEx', 'Robert Half Technology', 'UnitedHealth Group', 'Lockheed Martin', 'U.S. Army', 'The Home Depot', 'Duke Energy', 'GE Corporate', 'Modis', 'Apple', 'Capital One', 'CenturyLink', 'Lyft', 'Instacart Shoppers', 'HP', 'Postmates', 'Northrop Grumman', 'Honeywell', 'NASA', 'McKesson', 'Amazon Flex', 'Kaiser Permanente', 'Tradesmen International', 'Google', 'Peopleready', 'Boeing', 'Citi', 'CACI International Inc', 'Siemens AG', 'Pfizer Inc.', 'JPMorgan Chase', 'Shipt', 'U.S. Air Force', 'Amazon.com', 'IDC Technologies', 'FedEx Ground', 'Department of Homeland Security', 'UPS', 'Custom Communications, Inc.', 'Facebook', 'Johnson & Johnson', 'ExxonMobil']
['Weber Grill Restaurants, LLC', 'Steak N Shake', 'Compass Group', 'Bob Eva

['TAVO PACKAGING', 'UPS', 'Matheson Tri-Gas, Inc.', 'Amazon.com', 'Fastenal', 'ResourceMFG', 'FedEx', 'FedEx Ground', 'Pactiv', 'Graham Packaging Company', 'Zimmer Biomet', 'Saputo', 'The QTI Group', 'Bluegreen Vacations', 'OnStaff USA', 'Steel Technologies LLC', 'Acme Brick Company', 'IBM', 'Medline Industries', 'Cargill', 'Coca-Cola Beverages Florida, LLC', 'TERRA Staffing Group', 'Mancan', 'Greyhound', 'Mars', 'Conagra Brands', 'Unilever', 'Perrigo', 'KERRY', 'Westrock', 'BASF Corporation', 'MillerCoors', 'Henkel', 'Walmart', 'Curaleaf', "L'Oreal USA", 'Amazon Fulfillment Associates', 'Berlin Packaging', "Macy's", 'Tyson Foods Incorporated', 'Newell Brands', 'QUAD', 'Corporate Services, Inc.', 'General Mills', 'Wells Fargo', 'Tricorbraun, Inc.', 'Kraft Heinz Company', 'FurstStaffing', 'Associated Home Care, Inc.', 'Onward Search']
['Subway', 'Taco Bell', 'Walmart', 'McDonald LLC', "The Wendy's Company", 'Sonic Drive-In', 'Burger King', 'Starbucks', 'Whataburger', 'U.S. Air Force', '

['Snap Fitness', 'Subway - Mainly Manna, Inc.']
['AAON Inc', 'AAON', 'Aaon Coil Products', 'AAON, Inc', 'Aaon Coil', 'AAON, INC.']
['International Brotherhood of Teamsters', 'International Brotherhood of Teamsters Local']
[]
['Newco Inc', 'PCI Newco', 'Newco Manufacturing', 'Alfred Angelo, Newco', 'Newco Warranty', 'Hamlin Newco', 'Newco International', 'Newco Enterprises', 'Cameron NEWCO', 'NEWCO DISTRIBUTORS', 'Newco Enterprises Inc', 'NEWCO DESIGN BUILD', 'WMB NewCo', 'Newco Warranty Services', 'Tedeschi Food Shops', '7-Eleven', 'Jack in the Box', 'Danaher']
['Crom, LLC', 'Crom Construction', 'Crom Corporation', 'Crom LLC', 'The Crom Corporation', 'CROM EQUIPMENT RENTALS', 'CROM Engineering', 'CROM', 'Autometal']
['WILDER LLC', 'JOHN S. WILDER YOUTH DEVELOPMENT CENTER', 'Amherst H. Wilder Foundation', 'L. Douglas Wilder Library', 'ERA Wilder Realty', 'Michaels Wilder Inc', 'Wilder Foundation', 'WILDER CORPORATION', 'Wilder Elementary School', 'Wilder Ranch', 'Wilder Construction', '

...

['dayka & hackett llc']
['GMCO Corporation', 'GMCO']
[]
[]
['Giddy', 'Giddy Up', 'Giddy Up Glamour']
[]
['FCL Graphics']
['TruTeam', 'Innovative Refrigeration Systems', 'Installed Building Products', 'Brock Group', 'Zachry Group', 'KBR', 'Turner Industries', 'Performance Contracting Group', 'Jacobs', 'USI', 'Huntington Ingalls Industries', 'Wood Group', 'Tradesmen International', 'Craft and Technical Solutions, LLC', '84 Lumber Company', 'G5 Enterprises, Inc.', 'aptim', 'Mistras Group, Inc.', 'PLS Logistics Services', 'Service Partners', 'Electrical Talent, LLC', 'Mundy Companies', 'Nvent', "Josh Lowe's Dr. Energy Saver", 'Brown and Root Industrial Services', 'CLEAResult', 'ARS/Rescue Rooter', 'EMCOR', 'TopBuild', 'Safeway', 'BAE Systems', 'Masco', 'Zachry Construction Corporation', 'Fluor Corp.', 'Miller Insulation', 'CQ Insulation, Inc.', 'Isaac Heating & Air Conditioning', 'U.S. Navy', 'McDermott', 'BrandSafway Anacortes', 'National Lumber Family of Companies', 'Imerys', 'Quality In

['Dollinger Properties']
['Prima Power North America']
['GA Masonry']
[]
['Huawei']
['Microporous']
['ENVIRONMENTAL SYSTEMS INC', 'Environmental Air Systems', 'Otto Environmental Systems', 'Environmental Systems Design', 'Environmental Systems Research Institute', 'ANGUIL ENVIRONMENTAL SYSTEMS, INC.', 'Environmental Systems Products', 'Greensburg Environmental Contracting Systems Inc', 'Trap Zap Environmental Systems, Inc', 'ENVIRONMENTAL SERVICE SYSTEMS, LLC', 'Landmarc Environmental Systems', 'Environmental Systems Corporation', 'Environmental Monitoring Systems', 'Environmental Temperature Systems, LLC', 'DRS Environmental Systems', 'Carolina Environmental Systems.', 'Environmental & Fueling Systems, LLC', 'Environmental Systems Associates', 'Environmental Systems', 'Associated Environmental Systems', 'ENVIRONMENTAL CONTROL SYSTEMS', 'California Environmental Systems', 'Environmental Systems Corp', 'Environmental Service Systems', 'QED Environmental Systems, Inc', 'Carolina Environm

[]
['NOR-SON, Inc.']
['Peopleready', 'Tyson Foods Incorporated', 'Tradesmen International', 'CRH', 'Smithfield Foods', 'Mortenson Construction', 'Walmart', 'Army & Airforce Exchange Service', 'Perdue Farms, Inc.', 'Republic Services', 'Amazon.com', 'Goodwill Industries', 'PepsiCo', 'International Paper', 'Construction Labor Contractors', 'DHL', 'PACESETTER PERSONNEL SERVICES', 'CB&I', 'Cargill', 'Baker Roofing', 'Universal Forest Products, Inc.', 'Kraft Heinz Company', 'Mancan', 'Brightview', 'Turner Industries', '1-800-GOT-JUNK?', 'Staff Zone', 'Servpro', 'General Motors', 'Air Force Nonappropriated Fund (AFNAF) Services Career Program', 'UPS', 'MasTec Utility Services', 'Westrock', 'Blattner Energy, Inc.', 'ResourceMFG', 'The Home Depot', 'COLAS', 'Conagra Brands', 'General Mills', 'ARYZTA', 'ProLogistix', 'Target', 'Jacobs', 'Compass Group', '3M', 'Select Energy Services', 'Loram Maintenance of Way', 'Advantage Resourcing North America', 'CGB Enterprises Inc', 'JELD-WEN']
[]
['MILLE

['Secard Pools']
['Kahala Brands']
['ABM Industries, Inc.', 'The Hertz Corporation', 'C&W Services', 'Planet Fitness', 'Scioto Services', 'ServiceMaster', 'Avis Budget Group', 'Taco Bell', 'Enviro-Clean', 'Town Sports International', 'The Budd Group', 'The Cleaning Authority', 'IH Services, Inc', 'Motel 6 / Studio 6', 'GCA Services Group', 'Corporate Cleaning Group', 'KleenMark', 'Cushman & Wakefield', 'MaidPro', 'Enterprise Holdings', 'FBG Service Corporation', 'Marsden Building Maintenance', 'Topgolf', 'Janitorial Services, Inc.', 'ISS Facility Services', 'Nordstrom', 'SoulCycle', 'Marriott International, Inc.', 'Handy', 'Burger King', 'Walmart', 'Nelbud Services Group', 'B&T Contractors Inc', 'Mr. Rooter Plumbing', 'Swissport International Ltd.', 'Planned Companies', 'PrimeFlight Aviation', 'GRBS, Inc.', 'Kimco Services', 'Kellermeyer Bergensons Services', 'Two Maids & A Mop', "Buc-ee's", 'In-N-Out Burger', 'WinnCompanies', 'MasterCorp', 'Aramark', 'LA Fitness', 'Block By Block', 'E

['M&F Western Products']
['Johns Family Enterprises']
[]
['Distribution Technology', 'Honeywell', 'Carpenter Technology Corporation', 'Lenovo', 'World Wide Technology, Inc.', 'Arvato', 'Sole Technology']
['Heaven Hill Distilleries Inc', 'Heaven Hill Distilleries']
['LHC', 'LHC Group', 'LHC Homecare', 'Lhc Illinois Home Health Care', 'LHC AZ', 'LHC Physics Center', 'LHC/BAPTIST REYNOLDS', 'Walmart', 'JCPenney', 'Ochsner Health System', 'Enterprise Holdings', 'Addus HealthCare, Inc.', 'Cape Fear Valley Health System', 'Lafayette General Medical Center', 'University of Tennessee Medical Center']
['Nada Pacific Corporation']
['Southcreek Management Company', 'Subway', 'Taco Bell', 'Walmart', 'McDonald LLC', "The Wendy's Company", 'Sonic Drive-In', 'Burger King', 'Starbucks', 'Whataburger', 'U.S. Air Force', 'Little Caesars', "McDonald's", 'U.S. Army', 'KFC', 'UPS', 'Verizon', 'AT&T', 'U.S. Navy', "Lowe's", "Domino's", 'Walgreens', 'Deloitte', 'Chick-fil-A', "Papa John's", 'Sprint', 'Pizza 

In [65]:
pd.DataFrame({'company_name':company_name,'original_name':original_name,'review_link':review_link,'max_score':score_max}).to_excel('Dec_Renewal_indeed.xlsx')

## built condensed function for getting all matches

In [31]:
driver = get_browser()
url = "https://www.indeed.com/companies"

def get_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=3')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    return browser

def clean(name):
    name = name.lower()
    junk_words = [" corporation"," inc"," ltd"," pvt","."," llc"," llp","&amp",";",","]
    for word in junk_words:
        name = name.replace(word,"")
    name = name.strip()
    return name

def candidates_indeed_1(companyname,country,match_cutoff):
    driver = get_browser()
    url = "https://www.indeed.com/companies"

    driver.get(url)

    ## submitting name & location
    co_name = driver.find_element_by_xpath("""//*[@id="exploreCompaniesWhat"]""")
    co_name.send_keys(companyname.lower())

    location = driver.find_element_by_xpath("""//*[@id="exploreCompaniesWhere"]""")
    location.send_keys(country.lower())

    driver.find_element_by_xpath("""//*[@id="exploreCompaniesSearchFormContainer"]/form/div[3]/button""").click()

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:
        ### appending list of company names
        name =[]
        for l in soup.find_all("a",{"class":"cmp-CompanyWidget-name"}):
            name.append(l.text.strip())
#         print(name)
        
        ## storing list of companies fuzzy score with the query company name
        scores=[]
        found_names = []
        review_links = []
        position = 0
        for s in name:
            match_score = fuzz.ratio(clean(companyname).lower(),clean(s).lower()) 
            if match_score > match_cutoff:
                found_names.append(s)
                review_links.append("https://www.indeed.com"+soup.find_all("a",{"data-tn-element":"reviews-footer-link"},href=True)[position]['href'])
                scores.append(match_score)
            position+=1
        
    except:
        found_names = []
        review_links = []
        scores=[]  
    
    driver.refresh 
    return found_names, review_links, scores

    

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [33]:
candidates_indeed_1('Embassy Suites by Hilton','US',0)

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


(['Embassy Suites by Hilton',
  'Embassy Suites by Hilton OKC OU Medical Center',
  'Embassy Suites by Hilton Pittsburgh Downtown',
  'Embassy Suites by Hilton Minneapolis North',
  'Embassy Suites by Hilton Noblesville Indianapolis Conference Center',
  'Hilton',
  'Starbucks',
  'DoubleTree by Hilton',
  'Hilton Garden Inn Hotel'],
 ['https://www.indeed.com/cmp/Embassy-Suites-By-Hilton/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Okc-Ou-Medical-Center/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Pittsburgh-Downtown-1/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Minneapolis-North/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Noblesville-Indianapolis-Conference-Center-1/reviews',
  'https://www.indeed.com/cmp/Hilton/reviews',
  'https://www.indeed.com/cmp/Starbucks/reviews',
  'https://www.indeed.com/cmp/Doubletree-By-Hilton/reviews',
  'https://www.indeed.com/cmp/Hilton-Garden-Inn-Hotel/reviews'],
 [100, 69

## built condensed function for getting all matches (without using selenium, by forming the search url)

In [35]:
def candidates_indeed_2(companyname,country,match_cutoff):
    driver = get_browser()
## forming the url from company name and location    
    url = "https://www.indeed.com/cmp?q="+str(companyname)+"&l="+str(country)+"&from=discovery-cmp-search"
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:
        ### appending list of company names
        name =[]
        for l in soup.find_all("a",{"class":"cmp-CompanyWidget-name"}):
            name.append(l.text.strip())
#         print(name)
        
        ## storing list of companies fuzzy score with the query company name
        scores=[]
        found_names = []
        review_links = []
        position = 0
        for s in name:
            match_score = fuzz.ratio(clean(companyname).lower(),clean(s).lower()) 
            if match_score > match_cutoff:
                found_names.append(s)
                review_links.append("https://www.indeed.com"+soup.find_all("a",{"data-tn-element":"reviews-footer-link"},href=True)[position]['href'])
                scores.append(match_score)
            position+=1
        
    except:
        found_names = []
        review_links = []
        scores=[]  
    
    driver.refresh 
    return found_names, review_links, scores


In [38]:
candidates_indeed_2('Embassy Suites by Hilton','US',0)

C:\Users\arvind.baranwal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


(['Embassy Suites by Hilton',
  'Embassy Suites by Hilton OKC OU Medical Center',
  'Embassy Suites by Hilton Pittsburgh Downtown',
  'Embassy Suites by Hilton Minneapolis North',
  'Embassy Suites by Hilton Noblesville Indianapolis Conference Center',
  'Hilton',
  'Starbucks',
  'DoubleTree by Hilton',
  'Hilton Garden Inn Hotel'],
 ['https://www.indeed.com/cmp/Embassy-Suites-By-Hilton/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Okc-Ou-Medical-Center/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Pittsburgh-Downtown-1/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Minneapolis-North/reviews',
  'https://www.indeed.com/cmp/Embassy-Suites-By-Hilton-Noblesville-Indianapolis-Conference-Center-1/reviews',
  'https://www.indeed.com/cmp/Hilton/reviews',
  'https://www.indeed.com/cmp/Starbucks/reviews',
  'https://www.indeed.com/cmp/Doubletree-By-Hilton/reviews',
  'https://www.indeed.com/cmp/Hilton-Garden-Inn-Hotel/reviews'],
 [100, 69